# 事前準備

## ライブラリ

In [48]:
!pip install --upgrade xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import urllib.parse
import urllib.request
import json
import pandas as pd
import numpy as np
import xlrd

## estat-API

APIキーの設定

In [3]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

estat-APIからレスポンスを取得する関数

In [4]:
def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

レスポンスのJSONをDataFrameに整形する関数

In [5]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # 統計情報を追加
  df_res['statsDataId'] = TABLE_INF['@id']
  df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

# 取扱貨物量（総トン数）

## 2010年～2017年

[estat-API](https://www.e-stat.go.jp/dbview?sid=0003130803)からデータを取得する

In [6]:
params = {
    'statsDataId' : '0003130803',
    'cdCat02' : '100',
    'cdCat03' : '100',
}

res = get_estat_response(params)
print(res)

{'GET_STATS_DATA': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-09-22T14:09:41.376+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0003130803', 'NARROWING_COND': {'CODE_CAT02_SELECT': 100, 'CODE_CAT03_SELECT': 100}, 'DATA_FORMAT': 'J', 'START_POSITION': 1, 'METAGET_FLG': 'Y'}, 'STATISTICAL_DATA': {'RESULT_INF': {'TOTAL_NUMBER': 4510, 'FROM_NUMBER': 1, 'TO_NUMBER': 4510}, 'TABLE_INF': {'@id': '0003130803', 'STAT_NAME': {'@code': '00600280', '$': '港湾調査'}, 'GOV_ORG': {'@code': '00600', '$': '国土交通省'}, 'STATISTICS_NAME': '港湾統計（年報）', 'TITLE': {'@no': '2-3-1', '$': '第２部\u3000甲種港湾 第３表\u3000海上出入貨物表\u3000（１）トン数総数表 甲種港湾 2010年～'}, 'CYCLE': '年次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2018-02-06', 'SMALL_AREA': 0, 'MAIN_CATEGORY': {'@code': '10', '$': '運輸・観光'}, 'SUB_CATEGORY': {'@code': '01', '$': '運輸'}, 'OVERALL_TOTAL_NUMBER': 17380, 'UPDATED_DATE': '2022-07-29', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '港湾統計（年報）'}, 'TITLE_SPEC': {'TABLE_CATEGORY': '第２部\u3000甲種港湾', 'TABL

レスポンスのJSONをDataFrameに整形

In [7]:
df_res = conv_estat_response_to_dataframe(res)
df_res

,@tab,@cat01,@cat02,@cat03,@cat04,@time,@unit,$,@tab_name,@cat01_name,@cat02_name,@cat03_name,@cat04_name,@time_name,statsDataId,statsDataName
0,120,100,100,100,01000,2010000000,トン,190367576,トン数,合計,計,計,北海道,2010年,0003130803,港湾調査
1,120,110,100,100,01000,2010000000,トン,3641962,トン数,輸出,計,計,北海道,2010年,0003130803,港湾調査
2,120,120,100,100,01000,2010000000,トン,36090854,トン数,輸入,計,計,北海道,2010年,0003130803,港湾調査
3,120,130,100,100,01000,2010000000,トン,75494601,トン数,移出,計,計,北海道,2010年,0003130803,港湾調査
4,120,140,100,100,01000,2010000000,トン,75140159,トン数,移入,計,計,北海道,2010年,0003130803,港湾調査
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4588,NaN,NaN,NaN,NaN,46002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,鹿児島県_加治木,NaN,0003130803,港湾調査
4589,NaN,NaN,NaN,NaN,46004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,鹿児島県_喜入,NaN,0003130803,港湾調査
4590,NaN,NaN,NaN,NaN,46005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,鹿児島県_川内,NaN,0003130803,港湾調査
4591,NaN,NaN,NaN,NaN,46006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,鹿児島県_米之津,NaN,0003130803,港湾調査


In [8]:
# 必要な列だけ抽出
df_res =  df_res[['statsDataName','@cat01_name','@time_name','$','@unit','@cat04','@cat04_name']]

# 列名の変更
columns = {'@cat01_name':'種類','@time_name':'年次','@cat04':'港湾コード','@cat04_name':'港湾名','$':'取扱貨物量','@unit':'単位'}
df_res = df_res.rename(columns=columns)

# 欠損データ削除
df_res = df_res.dropna()

df_res

,statsDataName,種類,年次,取扱貨物量,単位,港湾コード,港湾名
0,港湾調査,合計,2010年,190367576,トン,01000,北海道
1,港湾調査,輸出,2010年,3641962,トン,01000,北海道
2,港湾調査,輸入,2010年,36090854,トン,01000,北海道
3,港湾調査,移出,2010年,75494601,トン,01000,北海道
4,港湾調査,移入,2010年,75140159,トン,01000,北海道
...,...,...,...,...,...,...,...
4505,港湾調査,合計,2017年,1051802,トン,47007,沖縄県_石垣
4506,港湾調査,輸出,2017年,2815,トン,47007,沖縄県_石垣
4507,港湾調査,輸入,2017年,40176,トン,47007,沖縄県_石垣
4508,港湾調査,移出,2017年,352868,トン,47007,沖縄県_石垣


In [9]:
df = df_res.copy()

# 取扱貨物量の欠損値を処理して数値に変換
df = df.replace('-', '0')
df = df.astype({'取扱貨物量': 'int'})

# クロス集計
df = pd.pivot_table(df, index=['港湾名','種類'], columns='年次',values='取扱貨物量',aggfunc=np.sum).reset_index()

df

年次,港湾名,種類,2010年,2011年,2012年,2013年,2014年,2015年,2016年,2017年
0,三重県,合計,NaN,NaN,64796525.0,62510340.0,63941182.0,NaN,NaN,NaN
1,三重県,移入,NaN,NaN,5770420.0,6431855.0,6695726.0,NaN,NaN,NaN
2,三重県,移出,NaN,NaN,15559712.0,13920666.0,15373861.0,NaN,NaN,NaN
3,三重県,輸入,NaN,NaN,39912016.0,38120375.0,38013776.0,NaN,NaN,NaN
4,三重県,輸出,NaN,NaN,3554377.0,4037444.0,3857819.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
600,鹿児島県_鹿児島,合計,40220057.0,39797738.0,39400674.0,39827495.0,38722283.0,34703113.0,34006495.0,34738544.0
601,鹿児島県_鹿児島,移入,20564919.0,20455814.0,20448680.0,20763653.0,20133788.0,17728699.0,17207308.0,17850130.0
602,鹿児島県_鹿児島,移出,18273515.0,17893985.0,17431160.0,17850846.0,17430003.0,15408797.0,15661639.0,15577484.0
603,鹿児島県_鹿児島,輸入,1373825.0,1447939.0,1509987.0,1208336.0,1153781.0,1550591.0,1101319.0,1292767.0


In [10]:
# CSVに書き出し
df.to_csv('全国港湾の取扱貨物量（合計）.csv',encoding='cp932')

## 2018～2020年

[国土交通省ホームページ](https://www.mlit.go.jp/k-toukei/kowannenpodb.html)のEXCELデータから作業する


2020年：https://www.mlit.go.jp/k-toukei/content/001448034.xlsx 

2019年：https://www.mlit.go.jp/k-toukei/content/001448577.xlsx

2018年：https://www.mlit.go.jp/k-toukei/content/001448540.xlsx


URLからEXCELデータを読み込んでDataFrameに格納

In [11]:
def get_dataframe(url):
  # ブックの指定
  input_file_name = url
  input_book = pd.ExcelFile(input_file_name)
  
  # 統計結果は2番目のシート、上から2行は無視
  input_sheet_name = input_book.sheet_names
  df_res = input_book.parse(input_sheet_name[1],skiprows = 2)

  return df_res

url = 'https://www.mlit.go.jp/k-toukei/content/001448034.xlsx'
df_2020 = get_dataframe(url)
df_2020

,都道府県,Unnamed: 1,港格,港湾,Unnamed: 4,種別,Unnamed: 6,合計,輸出,輸入,移出,移入
0,北海道,1,地方港湾,合計,NaN,一般,1.0,91743748,3045266.0,26759896.0,30127021.0,31811565.0
1,北海道,1,地方港湾,合計,NaN,自航,2.0,90269445,NaN,NaN,44205590.0,46063855.0
2,北海道,1,重要港湾,稚内,1002.0,一般,1.0,361697,33.0,1803.0,75040.0,284821.0
3,北海道,1,重要港湾,稚内,1002.0,自航,2.0,1073930,NaN,NaN,547890.0,526040.0
4,北海道,1,重要港湾,紋別,1048.0,一般,1.0,299245,1452.0,101281.0,71521.0,124991.0
...,...,...,...,...,...,...,...,...,...,...,...,...
320,沖縄県,47,重要港湾,運天,47002.0,自航,2.0,569520,NaN,NaN,285215.0,284305.0
321,沖縄県,47,重要港湾,平良,47003.0,一般,1.0,893011,3688.0,17360.0,129278.0,742685.0
322,沖縄県,47,重要港湾,平良,47003.0,自航,2.0,23275,NaN,NaN,20160.0,3115.0
323,沖縄県,47,重要港湾,石垣,47004.0,一般,1.0,726181,1963.0,17217.0,131313.0,575688.0


2010年～2017年と同型式になるよう、整形する

In [12]:
def format_dataframe(df_arg,time):
  df = df_arg.copy()

  # 都道府県と港湾で集計
  df = df.groupby(['都道府県','港湾']).agg({'合計': 'sum','輸出': 'sum','輸入': 'sum','移出': 'sum','移入': 'sum'})
  
  # 列の値を行にstack
  df = pd.DataFrame(df.stack()).reset_index()

  # 列名の整形
  df['港湾名'] = df.apply(lambda x: '{}_{}'.format(x['都道府県'],x['港湾']) if x['港湾'] != '合計' else x['都道府県'], 1)
  df[time] = df.apply(lambda x: x[0], 1)
  df['種類'] = df.apply(lambda x: x['level_2'], 1)

  # 必要な列だけ抽出
  df = df[['港湾名','種類',time]]
  
  return df

In [54]:
df_2020 = format_dataframe(df_2020,'2020年')
df_2020

KeyError: ignored

2010年～2017年と結合

In [14]:
df_res = pd.merge(df, df_2020, on=['港湾名','種類'] , how='left')
df_res

,港湾名,種類,2010年,2011年,2012年,2013年,2014年,2015年,2016年,2017年,2020年
0,三重県,合計,NaN,NaN,64796525.0,62510340.0,63941182.0,NaN,NaN,NaN,59851445.0
1,三重県,移入,NaN,NaN,5770420.0,6431855.0,6695726.0,NaN,NaN,NaN,6927958.0
2,三重県,移出,NaN,NaN,15559712.0,13920666.0,15373861.0,NaN,NaN,NaN,16240813.0
3,三重県,輸入,NaN,NaN,39912016.0,38120375.0,38013776.0,NaN,NaN,NaN,32932508.0
4,三重県,輸出,NaN,NaN,3554377.0,4037444.0,3857819.0,NaN,NaN,NaN,3750166.0
...,...,...,...,...,...,...,...,...,...,...,...
600,鹿児島県_鹿児島,合計,40220057.0,39797738.0,39400674.0,39827495.0,38722283.0,34703113.0,34006495.0,34738544.0,27746958.0
601,鹿児島県_鹿児島,移入,20564919.0,20455814.0,20448680.0,20763653.0,20133788.0,17728699.0,17207308.0,17850130.0,14203304.0
602,鹿児島県_鹿児島,移出,18273515.0,17893985.0,17431160.0,17850846.0,17430003.0,15408797.0,15661639.0,15577484.0,12309449.0
603,鹿児島県_鹿児島,輸入,1373825.0,1447939.0,1509987.0,1208336.0,1153781.0,1550591.0,1101319.0,1292767.0,1232606.0


2018～2020年について、これを繰り返せば良い

In [15]:
# 2018年
url = 'https://www.mlit.go.jp/k-toukei/content/001448540.xlsx'
df_2018 = get_dataframe(url)
df_2018 = format_dataframe(df_2018,'2018年')
df = pd.merge(df, df_2018, on=['港湾名','種類'] , how='left')

# 2019年
url = 'https://www.mlit.go.jp/k-toukei/content/001448577.xlsx'
df_2019 = get_dataframe(url)
df_2019 = format_dataframe(df_2019,'2019年')
df = pd.merge(df, df_2019, on=['港湾名','種類'] , how='left')

# 2020年
url = 'https://www.mlit.go.jp/k-toukei/content/001448034.xlsx'
df_2020 = get_dataframe(url)
df_2020 = format_dataframe(df_2020,'2020年')
df = pd.merge(df, df_2020, on=['港湾名','種類'] , how='left')

df

,港湾名,種類,2010年,2011年,2012年,2013年,2014年,2015年,2016年,2017年,2018年,2019年,2020年
0,三重県,合計,NaN,NaN,64796525.0,62510340.0,63941182.0,NaN,NaN,NaN,63860975.0,64335109.0,59851445.0
1,三重県,移入,NaN,NaN,5770420.0,6431855.0,6695726.0,NaN,NaN,NaN,7336766.0,7388105.0,6927958.0
2,三重県,移出,NaN,NaN,15559712.0,13920666.0,15373861.0,NaN,NaN,NaN,16093653.0,16508013.0,16240813.0
3,三重県,輸入,NaN,NaN,39912016.0,38120375.0,38013776.0,NaN,NaN,NaN,36208483.0,35795966.0,32932508.0
4,三重県,輸出,NaN,NaN,3554377.0,4037444.0,3857819.0,NaN,NaN,NaN,4222073.0,4643025.0,3750166.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,鹿児島県_鹿児島,合計,40220057.0,39797738.0,39400674.0,39827495.0,38722283.0,34703113.0,34006495.0,34738544.0,34840356.0,33045437.0,27746958.0
601,鹿児島県_鹿児島,移入,20564919.0,20455814.0,20448680.0,20763653.0,20133788.0,17728699.0,17207308.0,17850130.0,17801284.0,16853406.0,14203304.0
602,鹿児島県_鹿児島,移出,18273515.0,17893985.0,17431160.0,17850846.0,17430003.0,15408797.0,15661639.0,15577484.0,15615616.0,14814333.0,12309449.0
603,鹿児島県_鹿児島,輸入,1373825.0,1447939.0,1509987.0,1208336.0,1153781.0,1550591.0,1101319.0,1292767.0,1388136.0,1373673.0,1232606.0


# 取扱貨物量（品目別）

[国土交通省ホームページ](https://www.mlit.go.jp/k-toukei/kowannenpodb.html)のEXCELデータから作業する

輸出・輸入・移出・移入とも同じ構成のExcelになっているので、EXCELのインポートからDataFameの整形までは共通関数でOK

In [49]:
def get_excel(url):
  # ブックの指定
  input_file_name = url
  input_book = pd.ExcelFile(input_file_name)
  
  # 統計結果は2番目のシート、上から2行は無視
  input_sheet_name = input_book.sheet_names
  df = input_book.parse(input_sheet_name[1],skiprows = 2)

  return df

In [50]:
def get_dataframe(time,url):
 
  # EXCELデータをインポート
  df = get_excel(url)
  
  # 都道府県と港湾で集計
  df = df.astype({'トン数': 'int'})
  df = df.groupby(['都道府県','港名','品種名']).agg({'トン数': 'sum'}).reset_index()

  df = df[df['港名'] != '合計']

  df = df.rename(columns={'トン数':time})
  
  return df

繰り返し処理してひとつのDataFrameに整形する関数

In [51]:
def get_merge_dataframe(params):
  df_res = pd.DataFrame(index=[], columns=['都道府県','港名','品種名'])
  
  for p in params:
    df = get_dataframe(p['time'], p['url'])
    df_res = pd.merge(df_res, df, on=['都道府県','港名','品種名'] , how='outer')

  return df_res

## 輸出

In [52]:
params_export = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448464.xlsx' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448471.xlsx' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448478.xlsx' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448485.xlsx' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448499.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448521.xlsx' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448545.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448582.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448037.xlsx' },
]

In [53]:
df_export = get_merge_dataframe(params_export)
df_export

,都道府県,港名,品種名,2012年,2013年,2014年,2015年,2016年,2017年,2018年,2019年,2020年
0,三重県,四日市,その他日用品,1861.0,1376.0,1393.0,2405.0,2822.0,1466.0,2999.0,1384.0,1069.0
1,三重県,四日市,その他機械,3433.0,1340.0,371.0,239.0,107.0,522.0,107.0,NaN,616.0
2,三重県,四日市,その他畜産品,70.0,477.0,0.0,22.0,200.0,133.0,200.0,265.0,90.0
3,三重県,四日市,その他石油製品,5492.0,23263.0,36.0,98144.0,105.0,321.0,713.0,787.0,845.0
4,三重県,四日市,その他繊維工業品,2709.0,4575.0,2576.0,2118.0,2144.0,953.0,1532.0,84.0,476.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3482,鹿児島県,喜入,重油,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63800.0
3483,鹿児島県,川内,その他雑穀,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
3484,鹿児島県,志布志,完成自動車,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
3485,鹿児島県,志布志,米,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000.0


## 輸入

In [21]:
params_import = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/common/001277672.xls' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/common/001277688.xls' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/common/001277703.xls' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/common/001277717.xls' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448500.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/common/001277753.xls' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448546.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448583.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448038.xlsx' },
]

In [34]:
df_import = get_merge_dataframe(params_import)
df_import

,都道府県,港名,品種名,2012年,2013年,2014年,2015年,2016年,2017年,2018年,2019年,2020年
0,三重県,四日市,その他日用品,58191.0,80136.0,94268.0,122110.0,145715.0,189704.0,212454.0,183452.0,210129.0
1,三重県,四日市,その他林産品,1779.0,784.0,575.0,681.0,498.0,364.0,116.0,228.0,362.0
2,三重県,四日市,その他機械,0.0,NaN,0.0,57.0,57.0,63.0,1402.0,1063.0,82.0
3,三重県,四日市,その他石油製品,1103.0,6333.0,90.0,339.0,18.0,25709.0,168843.0,300259.0,420987.0
4,三重県,四日市,その他繊維工業品,3385.0,2389.0,2193.0,10240.0,6572.0,5883.0,7378.0,12979.0,10314.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4477,鳥取県,境,その他の石油,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9518.0
4478,鳥取県,境,その他輸送用車両,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175.0
4479,鹿児島県,喜入,重油,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171013.0
4480,鹿児島県,志布志,ＬＮＧ（液化天然ガス）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0


## 移出

In [42]:
params_emigration = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/common/001277673.xls' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/common/001277689.xls' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/common/001277704.xls' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/common/001277718.xls' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448501.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448522.xlsx' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448547.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448584.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448039.xlsx' },
]

In [43]:
df_emigration = get_merge_dataframe(params_emigration)
df_emigration

,都道府県,港名,品種名,2012年,2013年,2014年,2015年,2016年,2017年,2018年,2019年,2020年
0,三重県,四日市,その他石油製品,379629.0,846192.0,1017790.0,1220640.0,675150.0,610549.0,532937.0,543146.0,512377.0
1,三重県,四日市,とうもろこし,6446.0,35674.0,10000.0,5600.0,NaN,NaN,NaN,1770.0,1467.0
2,三重県,四日市,コークス,4500.0,18962.0,16550.0,42806.0,NaN,6900.0,7682.0,NaN,NaN
3,三重県,四日市,セメント,1256924.0,2749690.0,2790124.0,2014088.0,990337.0,900140.0,982981.0,964573.0,967494.0
4,三重県,四日市,分類不能のもの,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4989,鹿児島県,志布志,事務用機器,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
4990,鹿児島県,志布志,陶磁器,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
4991,鹿児島県,西之表,ゴム製品,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4992,鹿児島県,西之表,水,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## 移入

In [26]:
params_immigration = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/common/001277674.xls' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/common/001277690.xls' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/common/001277705.xls' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/common/001380154.xlsx' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/common/001380116.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448523.xlsx' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448548.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448585.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448040.xlsx' },
]

In [27]:
df_immigration = get_merge_dataframe(params_immigration)
df_immigration

,都道府県,港名,品種名,2012年,2013年,2014年,2015年,2016年,2017年,2018年,2019年,2020年
0,三重県,四日市,その他石油製品,38076.0,33310.0,1200.0,NaN,18.0,2202.0,105074.0,39662.0,36433.0
1,三重県,四日市,その他食料工業品,3811.0,1300.0,4399.0,6114.0,6139.0,5210.0,4054.0,3908.0,3847.0
2,三重県,四日市,とうもろこし,300.0,NaN,NaN,NaN,1500.0,NaN,3000.0,NaN,2400.0
3,三重県,四日市,コークス,39681.0,43442.0,21919.0,9330.0,6750.0,134848.0,4750.0,650.0,2900.0
4,三重県,四日市,セメント,50726.0,82441.0,56856.0,41049.0,34107.0,34522.0,42325.0,21240.0,40911.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5681,鹿児島県,名瀬,金属鉱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0
5682,鹿児島県,宮之浦(屋久島町),石灰石,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11020.0
5683,鹿児島県,志布志,事務用機器,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
5684,鹿児島県,西之表,鉄道車両,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0


## 搬出（輸出＋移出）

In [47]:
df_export = df_export.set_index(['都道府県','港名','品種名'])
df_emigration = df_emigration.set_index(['都道府県','港名','品種名'])

df_carryout = df_export + df_emigration
df_carryout.reset_index()
df_carryout

KeyError: ignored

## 搬入（輸入＋移入）

In [32]:
df_import = df_import.set_index(['都道府県','港名','品種名'])
df_immigration = df_immigration.set_index(['都道府県','港名','品種名'])

df_carryin = df_import + df_immigration
df_carryin

2012年     2013年     2014年     2015年     2016年  \
都道府県 港名  品種名                                                             
三重県  吉津  水産品               NaN       NaN       NaN       NaN       NaN   
     四日市 その他の石油            NaN       NaN       NaN       NaN       NaN   
         その他日用品            NaN       NaN   94336.0  122130.0  146211.0   
         その他林産品            NaN       NaN       NaN       NaN       NaN   
         その他機械             NaN       NaN       NaN       NaN       NaN   
...                        ...       ...       ...       ...       ...   
鹿児島県 鹿児島 非鉄金属           1831.0       NaN       NaN       NaN      43.0   
         飲料                NaN       NaN       NaN       NaN       NaN   
         麦            181322.0  218399.0  157008.0  188645.0  104550.0   
         ＬＮＧ（液化天然ガス）       NaN       NaN       NaN       NaN       NaN   
         ＬＰＧ（液化石油ガス）  130235.0  122799.0  121693.0  118812.0  112120.0   

                         2017年     2018年     2019年     2020年  
都道府県 港名  品種名                                                  
三重県  吉津  水産品               NaN       NaN       NaN       NaN  
     四日市 その他の石油            NaN  368068.0  374482.0  398453.0  
         その他日用品            NaN       NaN       NaN  210245.0  
         その他林産品            NaN       NaN       NaN       NaN  
         その他機械             NaN       NaN       NaN       NaN  
...                        ...       ...       ...       ...  
鹿児島県 鹿児島 非鉄金属             40.0      34.0       NaN      90.0  
         飲料                NaN       NaN       NaN       NaN  
         麦            140773.0  148985.0       NaN  149324.0  
         ＬＮＧ（液化天然ガス）       NaN       NaN       NaN       NaN  
         ＬＰＧ（液化石油ガス）  115763.0  110652.0  104679.0  100967.0  

[6858 rows x 9 columns]

# 入港船舶

In [55]:
params_ship = [
    { 'time' : '2012年' , 'url' : 'https://www.mlit.go.jp/common/001277666.xls' },
    { 'time' : '2013年' , 'url' : 'https://www.mlit.go.jp/common/001277682.xls' },
    { 'time' : '2014年' , 'url' : 'https://www.mlit.go.jp/common/001277697.xls' },
    { 'time' : '2015年' , 'url' : 'https://www.mlit.go.jp/common/001380148.xlsx' },
    { 'time' : '2016年' , 'url' : 'https://www.mlit.go.jp/common/001380113.xlsx' },
    { 'time' : '2017年' , 'url' : 'https://www.mlit.go.jp/common/001380092.xlsx' },
    { 'time' : '2018年' , 'url' : 'https://www.mlit.go.jp/common/001380059.xlsx' },
    { 'time' : '2019年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448575.xlsx' },
    { 'time' : '2020年' , 'url' : 'https://www.mlit.go.jp/k-toukei/content/001448031.xlsx' },
]